In [14]:
from selenium import webdriver
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox) # linux only
chrome_options.add_argument("--headless")

from bs4 import BeautifulSoup as bs
import os
import re
import twitter_keys as tweet
import time
import pickle

In [4]:
def pull_tweets(soup):
    """
    Helper function that grabs tweets using BeautifulSoup
    """
    results = soup.find('ul')
    results = results.findAll('li')
    return [tweet.text for tweet in results]

You will need to create a seperate called 'twitter_keys.py' and save it int he same directory.

In that file add two lines with your account information:

account = 'twitter_handle'

account_password = 'twitter_password'

In [12]:
def tweet_collector(unsub):
    chrome_path = r"C:\Programs\Python\Python37-32\Scripts\chromedriver_win32\chromedriver.exe"
    driver = webdriver.Chrome(chrome_path, options=chrome_options)

    URL = 'https://www.allmytweets.net/'
    driver.get(URL)

    driver.find_element_by_id('connect-btn').click()

    username = driver.find_element_by_id("username_or_email")
    username.clear()
    username.send_keys(tweet.account)

    password = driver.find_element_by_id("password")
    password.clear()
    password.send_keys(tweet.account_password)

    driver.find_element_by_id('allow').click()

    twitter_user = driver.find_element_by_id('input-name')
    twitter_user.clear()
    twitter_user.send_keys(unsub)

    driver.find_element_by_id('load-user-btn').click()

    driver.implicitly_wait(10)
    time.sleep(5)

    try:
        driver.find_element_by_id('tweets-btn').click()
    except: 
        driver.find_element_by_id('tweets-btn').click()
    
    time.sleep(5)
    
    soup = bs(driver.page_source, 'lxml')

    
    tweet_list = pull_tweets(soup)
    
    total_tweets = None
    try:
        total_tweets = soup.find(id='load-status').text
        if len(total_tweets) > 1:
            total_tweets = re.findall('[\d]+', total_tweets)[0]
        else:
            total_tweets = 'Number of tweets not found'
    except:
        time.sleep(10)
        total_tweets = soup.find(id='load-status').text
        if len(total_tweets) > 1:
            total_tweets = re.findall('[\d]+', total_tweets)[0]
        else:
            total_tweets = 'Number of tweets not found'

    try:
        driver.quit()
    except:
        driver.close()
    
    if total_tweets != None:
        print('User: {} \nExpected tweets: {}\nTweets Collected: {}\n'.format(unsub, total_tweets, len(tweet_list)))
        
    return tweet_list

Insert Twitter Handle in list 'unsubs' below

In [13]:
unsubs = []

In [10]:
def update_tweet_dict(unsubs):
    """
    Creates a dictionary with twitter handles as keys and tweets and values
    """
    tweet_dict = {}
    for user in unsubs:
        if user not in tweet_dict.keys() or tweet_dict[user] == []:
            tweets = tweet_collector(user)
            tweet_dict[user] = tweets
        else:
            continue
    return tweet_dict